In [1]:
import re
import csv
from funcs import *

In [10]:
def find_indels(char, string):
    global indexes
    position = string.find(char) + 1
    indexes.append(position)
    new_string = string[position + len(char)-1:]
    if char in new_string:
        return find_indels(char, new_string)
    if len(indexes) > 1:
        for i in range(1, len(indexes)):
            indexes[i] += indexes[i-1] + len(char) - 1

Переходим в папку с BAM-файлом

In [7]:
cd PATH_TO_BAM_FILE

/Users/macbookair/20210610_g170_pool/124


### Считаем HDR

Запускаем программу pysamstats, указав ей путь к референсу и BAM-файл

In [8]:
!pysamstats -f PATH_TO_REFERENCE --type variation \
    --max-depth=50000 BAM_FILE.bam | grep -E "3387|chrom" | cut -f2,3,4,16 > hdr_stats.txt

with open('hdr_stats.txt') as hdr:
    next(hdr)
    for line in hdr:
        number_HDR = int(line.split()[-1])
        percent_HDR = str(round(int(line.split()[-1]) / int(line.split()[-2]) * 100, 3)) + '%'

<frozen importlib._bootstrap>:219: RuntimeWarning: pysam.libcalignedsegment.PileupColumn size changed, may indicate binary incompatibility. Expected 72 from C header, got 88 from PyObject


### Считаем инделы

Запускаем команду, выбрав соответствующий BAM-файл и поменяв название выходного файла:

In [ ]:
!samtools view sorted_124.bam | cut -f1,4,6,9 | sort -o CIGARS.tsv

Запускаем скрипт по поиску инделов, заменив, если нужно, название файла в первой строчке

In [16]:
with open('CIGARS.tsv') as read_file:
    tsvreader = csv.reader(read_file, delimiter="\t")
    read_list = [line for line in tsvreader]


filtered_dict = dict()
for read in read_list:
    if read[0] not in filtered_dict.keys():
        filtered_dict[read[0]] = read[1:4]
    else:
        filtered_dict[read[0]] += read[1:4]
        

reads_forward, reads_reverse = list(), list()
for read in read_list:
    if int(read[3]) > 0:
        reads_forward.append(decode_CIGAR(string_code=read[2], starting_position=read[1]))
    else:
        reads_reverse.append(decode_CIGAR(string_code=read[2], starting_position=read[1]))

        
ROI_forward = [select_region(i[1], i[0]) for i in reads_forward]
ROI_reverse = [select_region(i[1], i[0]) for i in reads_reverse]


final_ROIs = dict()
for string in ROI_forward:
    if string[1] != '':
        if string[1] not in final_ROIs.keys():
            final_ROIs[string[1]] = [1, string[0]]
        else:
            final_ROIs[string[1]][0] += 1
for string in ROI_reverse:
    if string[1] != '':
        if string[1] not in final_ROIs.keys():
            final_ROIs[string[1]] = [1, string[0]]
        else:
            final_ROIs[string[1]][0] += 1

            
STATS = {}
D_3393 = int()


for key, value in final_ROIs.items():
    if re.search(r'[A-Z]', key) != None:
            indels = set(indel for indel in re.findall(r'\.+(D+|I+)', key) if indel != '')
            
            for i in indels:
                indexes=[]
                find_indels('.' + i + '.', key)
                indexes = [j + value[1] for j in indexes]
                
                for j in indexes:
                    if 3378 <= j < 3387 or j < 3378 and j + len(i) >= 3383: 
                        new_key = i + '_' + str(j)
                        if new_key not in STATS.keys():
                            STATS[new_key] = value[0]
                        else:
                            STATS[new_key] += value[0]
                    elif j == 3393:
                        if i == 'D':
                            D_3393 += value[0]
STATS = sort_dict(STATS)


total_reads = 0
for n in final_ROIs.values():
    total_reads += int(n[0])

    
with open('OUTPUT.csv', 'w') as output:
    output.write(str('Header;Count;Percent;'+'\n'))
    output.write(str('Total number of reads;' + str(total_reads)+';'+'\n'))
    output.write(str('Indels in ROI;'+str(sum(STATS.values()))+';'+str(round(sum(STATS.values())/total_reads * 100, 3)))+'\n')
    output.write(str('HDRs;' + str(number_HDR) + ';' + str(percent_HDR)+'\n'))